In [1]:
import regex as re
import numpy as np
import pandas as pd
import os
from time import time
import matplotlib.pyplot as plt
import pythainlp

In [2]:
os.getcwd()

'/home/kamin/Thai_NLP'

In [3]:
os.chdir('Thai_NLP')

FileNotFoundError: [Errno 2] No such file or directory: 'Thai_NLP'

In [3]:
from preprocessing import Word_Embedder
from preprocessing import Text_processing
from text_classification import Text_classification, Text_classification_for_prediction, count_based_model
from text_classification import prepare_data_for_text_classification

In [9]:
a = 'ฉันไม่ชอบ55เลย5'
pattern = re.compile("5")
m = pattern.finditer(a)
# m = re.match('5', a)
for i in m:
    print(i.start())
    print(i.end())
    print('---')

9
10
---
10
11
---
14
15
---


In [10]:
def concordance(
    pattern,
    expand_size: '(int) expand to the left and right with this size',
    text: '(str) text to be searched'
):
    pattern = re.compile(pattern)
    tmp = []
    for i in pattern.finditer(text):
        st = max(0,i.start() - expand_size)
        en = min(i.end() + expand_size, len(text))
        tmp.append(text[st:en])
    return tmp

In [14]:
x = [
    'ไก่ฉันกินข้าวมันไก่',
    'ไม่ค่อยไก่นะพี่',
    'ไก่รอบนี้ไม่อร่อยเลย'
    ]

In [16]:
for i in x:
    print(concordance('ไก่',5,i))

['ไก่ฉันกิ', 'าวมันไก่']
['่ค่อยไก่นะพี่']
['ไก่รอบนี']


In [4]:
RS = 123
tr = pd.read_csv(f'train_RS_{RS}.csv')
te = pd.read_csv(f'test_RS_{RS}.csv')
tr = tr.dropna().reset_index(drop=True)
te = te.dropna().reset_index(drop=True)

In [28]:
os.chdir('word_configs')

In [29]:
neg = set()
with open('negative.txt', 'r', encoding='utf8') as f:
    for i in f:
        i = i.strip()
        if i != '':
            neg.add(i)
neu = set()
with open('neutral.txt', 'r', encoding='utf8') as f:
    for i in f:
        i = i.strip()
        if i != '':
            neu.add(i)

In [31]:
z = neg.intersection(neu)
print(z)

{'แต่', 'โอเค', 'บอก', 'ไม่', 'ว่า', 'นี้', 'อะไร', 'ต้อง', 'จะ', 'ผม', 'ให้', 'แบบ', 'ของ', 'คน', 'ๆ', 'เรื่อง', 'การ', 'ได้', 'เรา', 'เลย', 'ที่', 'ทำ', 'มัน', 'มี', 'คะ', 'บัตร', 'ถ้า', 'ยัง', 'แล้ว', 'เป็น'}


In [35]:
for pat in z:
    print(f'pat: {pat}')
    print(pythainlp.tokenize.word_tokenize(pat,engine='deepcut'))
    expand_size = 6
    dictionary = {}
    for i in tr['texts']:
        for j in concordance(pat, expand_size, i):
            if j not in dictionary:
                dictionary[j] = 1
            else:
                dictionary[j] += 1
    a = sorted(dictionary.items(), key = lambda x: x[1], reverse=True)
    for i in a:
        if i[1] >1:
            print(i)
    print('-------------------------------')

pat: แต่
['แต่']
('ก็แล้วแต่', 4)
('แต่ตอนนี้', 2)
('แต่พอจะยก', 2)
('็นตั้งแต่เรื่อง', 2)
-------------------------------
pat: โอเค
['โอเค']
('ี่ยมันโอเคหรือเป', 2)
('หนูพูดโอเค รู้เร', 2)
-------------------------------
pat: บอก
['บอก']
-------------------------------
pat: ไม่
['ไม่']
('งก็ยังไม่ได้รับ', 4)
('ำเสียงไม่เต็มใจ', 3)
('ไม่ว่าจะเ', 2)
('ยังไม่ได้รับ', 2)
('คุณไม่เข้าใจ', 2)
('ไม่ทางไปร', 2)
('ไม่ประทับ', 2)
('ไม่มีอะไร', 2)
('้น คุณไม่รับผิด', 2)
('ในความไม่สะดวกท', 2)
('ไม่ต่างกั', 2)
('อนอะไรไม่ได้', 2)
('้ไขให้ไม่ได้', 2)
('นเตอร์ไม่ได้', 2)
('ผ่อนก็ไม่ได้แย่', 2)
('ไม่ต้องโท', 2)
-------------------------------
pat: ว่า
['ว่า']
('รู้สึกว่ามีความ', 2)
('ไม่ว่าจะเป็น', 2)
('เขาบอกว่า', 2)
('าเรียกว่าอะไรนะ', 2)
('งเพราะว่า', 2)
('้องบอกว่า', 2)
-------------------------------
pat: นี้
['นี้']
('ป็นแบบนี้', 3)
('ล้ววันนี้', 3)
('ประมาณนี้', 3)
('ะนำแบบนี้ด้วย ร', 2)
('รอย่างนี้นะครับ', 2)
('อันนี้ถือว่า', 2)
('งินตอนนี้', 2)
('รณ์แบบนี้ขึ้น ค', 2)
('อันนี้จะเป็น', 2)
('อ

In [38]:
pythainlp.tokenize.word_tokenize('ไม่คุ้มเลย',engine='deepcut')

['ไม่', 'คุ้ม', 'เลย']

In [39]:
for i in tr[tr['labels'] == 'negative']['texts']:
    print(i)

ไม่มีค่ะ
คุณทำอะไรได้บ้าง
หัวหน้าอยู่ไหม
เบื่อมากเหมือนกัน เช็คอะไรก็ไม่ได้
กังวลมากค่ะ
จึงไม่สามารถทำให้คนเข้าใจอะไรด้วยข้อความไม่กี่ข้อความ
คุณจบอะไรมา
แทนที่จะ พิจารณา ตัวเอง
นี้คือการหลอกลวงชัดๆ แย่มากๆ
บอกว่าจะติดตามให้ แล้วก็หายจ๋อม
ขาดประสิทธิภาพในการทำงาน
งี้เง่า พอได้แล้ว นี่จะกวนประสาทใช่ป่ะ
แล้วบอกให้ผมไปฟ้องร้องเอา(แบบนี้ก็ได้หรอ) นี้คือการทำงานกันแบบนี้ใช่ไหม
cs hold สาย 0716-1734 น ประมาณ 10 น
รำคาญตรงเงื่อนไขที่บางไตรมาสจุกจิกเกิน
ถ้าเราไม่เช็คก็เหมือนแต้มเราหายไปเฉยๆ
ไม่คุ้มเลย
คุยได้ความว่าจะตามเรื่องให้แล้วติดต่อกลับ ก็หายไป
พอทยอยจ่าย ก็กลายเป็นตัดมั่ว
พอทำเรื่องขอคืนสินค้า กลับถูกบ่ายเบี่ยง
ได้จดหมายปฏิเสธเห็นเหตุผลแล้วพิศดาร งงมาก
รู้สึกไม่มีความรับผิดชอบยังไงไม่รู้
อะไรวะ โทรไปก็ให้รอสาย เสียค่าโทรศัพทอีก ไม่มีคนรับ ให้ทำยังไง
ไม่ชอบ
เดี๋ยวก่อนนะ สรุปคุณทำอะไรให้ได้บ้างไหม ฟังฉันพูดให้จบก่อนก่อนจะพูดแทรกขึ้นมา คุณชื่ออะไรนะคะ คุณยังฟังลูกค้าพูดไม่จบเลย
บริษัทคุณควรจะอบรมพนักงานให้ดีมากกว่านี้ จะไม่ได้เกิดเหตุการณ์กับลูกค้าท่านอื่นอีก
พอแจ้งยกเลิกไปก็ยอกมห้รอเจ้าหน

In [5]:
TC = Text_classification_for_prediction( path_to_tfxidf= 'models_for_real_deployment/tf-idf_encoder.joblib',
                                        model_path = 'models_for_real_deployment/text_classification',
                                       engine='linear_classifier')

loading logistic regression model


In [9]:
for (_,i) in tr.iterrows():
    tmp = TC.predict(i.texts)
    if tmp['logistic_regression']['negative'] > 0.5 and i.labels != 'negative': 
        print(f'text: {i.texts} ==> Negative')

text: แล้วก็มาขายกันเร็ว ==> Negative
text: แล้วแต่ lifestyle จริงๆ ครับ ==> Negative
text: ความน่าสนใจก็คือเงินเหล่านั้นที่มันโปรยลงมาเนี่ยมันโปรยลงมา ==> Negative
text: คุณจะมีเหมือนกับคำนำหน้านะครับ ==> Negative
text: กลับครับผมกับแฟนคนก่อนที่เป็นชาวต่างชาติครับผม ==> Negative
text: อสังหาริมทรัพย์บางทีก็จะมีราคาขึ้นลงได้บ้างนิดนึงคือความฝัน ==> Negative
text: วิธีแก้ปัญหาครับปิดแอปลบไปเลยดีกว่าครับช่วงบ่ายผมมีมีช่วงนึงใส่เพิ่ม ==> Negative
text: เช่นหลังจากเจอช่วง covid ครับเรื่องการบริหารการเงินส่วนบุคคลของ ==> Negative
text: คุยกับนักธุรกิจทุกคนครับ ==> Negative
text: ทำไมมันสวนทางกัน ==> Negative
text: มีกันแค่สองคนแล้วก็อยากให้พ่อได้รับรู้ไว้ว่าลูกวางแผนอะไรไว้บ้าง ==> Negative
text: มาครับรายการนะครับเป็นรายการที่จะ ==> Negative
text: สวัสดีครับ ==> Negative
text: สัญญาโอเคแหละครับ แต่ไม่น่ามีผลอะไรหรอก ==> Negative
text: ลงมาถึงเลย ==> Negative
text: สินเชื่อประเภทนี้ก็คือสินเชื่อรถเนี่ยมันไม่ได้เป็นรถต้นลดดอก ==> Negative
text: อยากจะใช้สิทธิประโยชน์ทางภาษีด้วยอยากจะรู้เรื่อ

text: นึกว่าไม่เกิดเมาฉิบหาย effect ไม่เกิดการใช้เงิน ==> Negative
text: ทั้งนี้ต้องขออภัยในความไม่สะดวกที่เกิดขึ้นด้วยนะคะ อย่างไรรบกวนแจ้ง ชื่อ และ นามสกุล ==> Negative
text: แอบซ่อนอย่างเดียวแต่ก็ต้องแอบที่จะสะกิดบอก ==> Negative
text: พูดตรงๆค่อนข้างแข็งแกร่งเพียงแต่ ==> Negative
text: ได้เลยได้เลยครับพี่ใช้ได้ใช้ได้เลยนะครับแล้วก็แม่ ==> Negative
text: ไม่ทราบเหมือนกันครับ ==> Negative
text: แต่ต้องบอกจริงว่าไม่กล้าแนะนำว่าอะไรมันจะพุ่ง ==> Negative
text: จากที่เราต้องมาลุ้นว่าอันนี้มันเป็นคำถามแนวไหน ==> Negative
text: มันไปว่าผลกระทบเป็นยังไงก็ลองไปคำนวณเอาเองผมก็ไม่รู้ว่ามันบวกหรือลบ ==> Negative
text: ถามจริงตลาดนะเนี่ยมันเป็นตลาดที่สกุลศึกษามันดีๆ ==> Negative
text: ก็ไปด่าเพื่อน ==> Negative
text: อิเล็กทรอนิกส์เนี่ยมันมา ==> Negative
text: ชีวิตเลยครับจริงๆถ้าปล่อยให้นานมากขึ้น ==> Negative
text: บวชด้วยคือผมก็อยู่อีกกระบวนหนึ่งจะเป็นสไตล์หวานเย็น ==> Negative
text: คุณถือหุ้นเนี่ย ==> Negative
text: ครับกลับก็ลองดูอายุ 28 เองครับแล้วก็ ==> Negative
text: ก็เลยทำให้ที่โรงแร

text: ด้วยตัวเอง จะได้ไม่เสียใจเสียเวลาเสียสุขภาพจิต เพราะพลาดคนอื่น ==> Negative
text: มาทยอยเหมือนกับยาย ==> Negative
text: อะไรของคาดว่าจะโตวัน ==> Negative
text: คุณพ่อบอกว่า ==> Negative
text: เช่นกันครับผม ==> Negative
text: ถูกหรือไม่คะ ==> Negative
text: แต่ละประเทศก็จะทำไม่เท่ากัน ==> Negative
text: สวัสดีครับผมใช่ครับก็ต้องอาศัยเขาเรียกว่าความรอบคอบระมัดระวังอยู่พอสมควรครับเพราะว่าจริงๆแล้ว ==> Negative
text: อาหารไม่เน้นมาก บินไฟลท์ เที่ยงคืนครับ ==> Negative
text: ทางฝั่งยุโรปอเมริกาจะดู ==> Negative
text: ผลกำไรผลประกอบการเขาก็ไม่ได้สวยหรูอะไรมากก็ยังปกติแต่ ==> Negative
text: ประหยัดดอกเบี้ยไหมครับผมคิดถูกหรือเปล่า ==> Negative
text: ดูภาวะตลาดร่วมด้วยอย่างนู้นอย่างนี้อะไรอย่างนี้บ้างลงทุนเป็นแบบแพ็ค ==> Negative
text: มีตัวนี้รอดูอยู่นะอะไรเงี้ยก็จะมีอยู่บ้าง ==> Negative
text: แล้วพบกันใหม่ตอนหน้าครับวันนี้ลาไปก่อนนะครับสวัสดีครับสวัสดีครับติดตามทุกรายการของ The Standard ==> Negative
text: มันไม่ไปถึง ==> Negative
text: สัมภาษณ์เรื่องอะไรบ้างครับผมว่าจุดร่วมสำคัญที่สุดก

text: ประเด็นมีปัญหาทางด้านการเงินอยู่ครับ ==> Negative
text: ถามอย่างของ ==> Negative
text: พอร์ตการลงทุนของผม ==> Negative
text: ขอบคุณครับการติดตามครับลาไปก่อนแล้วพบกันในวันนี้ ==> Negative
text: ครับผม ==> Negative
text: บริษัทเนี่ยไม่เข้าข่าย esc นะ ==> Negative
text: รองเท้านี่เป็นตั้งเลยครับเป็นตำราเลย ==> Negative
text: และไอ้สรุปอันนี้พอทำเป็นงก ==> Negative
text: เริ่มหาทางแก้อะไรนะตรงไหนเป็นทรัพย์สินก็บอกกันไว้ให้รู้กันไว้ทั้งสองฝ่าย ==> Negative
text: แล้วก็ล่าสุดนี้ SF ==> Negative
text: เข้ามาเป็นนักลงทุนเต็มตัวนั้นผมก็มีอะไรก็จะคุยกันว่าตอนนี้คืออะไรอยู่ทำอะไรอยู่นะครับแต่ว่าส่วนใหญ่ ==> Negative
text: ภาพรวมของตลาดก่อนตอนนี้มันมี ==> Negative
text: ฟิวชั่นเนียมน้องไปอ่านตรงนโยบายการลงทุนใช่ไหมคะเขียนว่า ==> Negative
text: สวัสดีค่ะโค้ชหนุ่มพี่โอมสบายดีไหมคะสบายดีครับสบายดีครับ ==> Negative
text: และผม ก็ไม่ไดทำสัญญา กู้เงิน เอาไว้ ผมจึ่ง ราง หนังสือสัญญารับสภาพนี้ ==> Negative
text: สีใช้ได้หลายตัวนะครับผม ==> Negative
text: จะต้องบอกอย่างนี้ก่อนเนาะ ==> Negative
text: 